In [103]:
import subprocess, json
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import date

In [27]:
skus_df = pd.read_csv("outputs/skus.csv")
skus = skus_df.idMedicamento.astype(str).unique().tolist()

In [92]:
import requests
import json

def get_data(skus):
  url = "https://medicamentos.fonasa.cl/buscador-farmacias/buscarFarmacias"

  try:
    payload = json.dumps({
      "longitud": -70.6216794,
      "latitud": -33.4843354,
      "medicamento": ",".join(skus),
      "distancia": 50
    }).replace(" ","")
    headers = {
      'Content-Type': 'application/json'
    }   

    response = requests.request("POST", url, headers=headers, data=payload)

    temporal_df = pd.DataFrame(response.json().get("medicamento").get("medicamento"))
    return temporal_df
  except:
    print("error in:", skus)

list_of_dfs = []

BATCH_SIZE = 10 
items = len(skus)
BATCHES = int(np.ceil(items / BATCH_SIZE))

for i in tqdm(range(BATCHES)):
  start = i * BATCH_SIZE
  end = (i + 1) * BATCH_SIZE
  mini_batch_skus = skus[start:end]
  mini_batch_df = get_data(mini_batch_skus)
  list_of_dfs.append(mini_batch_df)
  



100%|██████████| 498/498 [05:21<00:00,  1.55it/s]


In [1]:
(
    pd.concat(list_of_dfs)
    .drop(columns=["nombreRegion"])
    .assign(fecha_proceso=date.today())
    .to_csv("outputs/data_medicamentos_fonasa.csv", index=False, sep=";")
)

NameError: name 'pd' is not defined